<a href="https://colab.research.google.com/github/ArmstrongVo/VoTrongLuan.github.io/blob/main/model_VGGL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# import libraries
import keras
import cv2
import tensorflow as tf
from keras.models import Sequential
from glob import glob
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, Conv3D, BatchNormalization
from keras.layers import LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img,img_to_array, array_to_img
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model


In [6]:
# access to data
test='/content/drive/MyDrive/DATA FOR AI/MV_DATA/test2'
train='/content/drive/MyDrive/DATA FOR AI/MV_DATA/train2'


In [7]:
# để hệ thống đạt được độ chính xác cao nhất
# tiến hành pre-processing sử dụng kỹ thuật data agumentation
# thiết lập các thông số cho data augmentation
# xoay ảnh
# lật chiều ngang
# lật dọc
# phóng to hoặc thu nhỏ

train_augen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=0.2, 
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2)

test_augen = ImageDataGenerator(rescale=1./255)

train_set = train_augen.flow_from_directory(train,
                                            target_size=(224, 224),
                                            batch_size=64,
                                            class_mode='categorical')
                                               
test_set = test_augen.flow_from_directory(test,
                                          target_size=(224, 224),
                                          batch_size=64,
                                          class_mode='categorical')

Found 14882 images belonging to 3 classes.
Found 301 images belonging to 3 classes.


In [ ]:
# cấu trúc CNN
model = Sequential()
# sử dụng 16 bộ lọc kích thước 3*3 để tạo tích chập
# trọng số " he-uniform"
# tạo bộ đệm để đầu ra giống đầu vào
# hạn chế mất mát dữu liệu khi tích chập có giá trị âm
# pooling các giá trị lớn nhát trong vùng 2*2 
model.add(Conv2D(32, kernel_size=(3,3), activation='relu',kernel_initializer='he_uniform', padding='same',  input_shape=(224, 224,3)))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())  # chuyển đầu ra 2D thành  1D để đưa vào lớp fullconnected
model.add(Activation("relu"))
model.add(Dense(512))
model.add(Dropout(0.4)) # loại bỏ bớt 1 số thành phần để tránh ovefitting_ không tổng quát tốt được quá trình học
model.add(Dense(3))  # fullconnected 3 ngõ ra 
model.add(Activation("softmax"))   # đưa ra xác suất dự đoán cho mỗi lớp

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                    

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training_model = model.fit(train_set,
                    steps_per_epoch=200,
                    epochs=50,
                    validation_data=test_set,
                    validation_steps=5)
model.save("/content/drive/MyDrive/DATA FOR AI/MV_DATA/VGG.h5")

Epoch 1/10
200/200 [==============================] - 3006s 15s/step - loss: 3.1891 - accuracy: 0.6908 - val_loss: 0.8529 - val_accuracy: 0.5781
Epoch 2/10
200/200 [==============================] - 566s 3s/step - loss: 0.5679 - accuracy: 0.7426 - val_loss: 0.7090 - val_accuracy: 0.6279
Epoch 3/10
200/200 [==============================] - 243s 1s/step - loss: 0.5049 - accuracy: 0.7781 - val_loss: 0.6388 - val_accuracy: 0.7176
Epoch 4/10
200/200 [==============================] - 193s 966ms/step - loss: 0.4514 - accuracy: 0.8051 - val_loss: 0.6887 - val_accuracy: 0.6910
Epoch 5/10
200/200 [==============================] - 189s 943ms/step - loss: 0.4044 - accuracy: 0.8330 - val_loss: 0.6982 - val_accuracy: 0.6346
Epoch 6/10
200/200 [==============================] - 186s 927ms/step - loss: 0.3804 - accuracy: 0.8407 - val_loss: 0.4440 - val_accuracy: 0.7940
Epoch 7/10
200/200 [==============================] - 186s 930ms/step - loss: 0.3467 - accuracy: 0.8568 - val_loss: 0.4760 - val_ac

In [4]:
model=load_model('/content/drive/MyDrive/DATA FOR AI/MV_DATA/VGG.h5')
training_model = model.fit(train_set,
                    steps_per_epoch=100,
                    epochs=20,
                    validation_data=test_set,
                    validation_steps=5)
model.save("/content/drive/MyDrive/DATA FOR AI/MV_DATA/VGG_1.h5")

Epoch 1/20
100/100 [==============================] - 795s 8s/step - loss: 0.2776 - accuracy: 0.8897 - val_loss: 0.6329 - val_accuracy: 0.7409
Epoch 2/20
100/100 [==============================] - 341s 3s/step - loss: 0.2950 - accuracy: 0.8782 - val_loss: 0.4836 - val_accuracy: 0.7674
Epoch 3/20
100/100 [==============================] - 237s 2s/step - loss: 0.2901 - accuracy: 0.8822 - val_loss: 0.3876 - val_accuracy: 0.8538
Epoch 4/20
100/100 [==============================] - 181s 2s/step - loss: 0.2771 - accuracy: 0.8895 - val_loss: 0.6358 - val_accuracy: 0.7575
Epoch 5/20
100/100 [==============================] - 136s 1s/step - loss: 0.2807 - accuracy: 0.8867 - val_loss: 0.6306 - val_accuracy: 0.7276
Epoch 6/20
100/100 [==============================] - 121s 1s/step - loss: 0.2671 - accuracy: 0.8918 - val_loss: 0.6059 - val_accuracy: 0.7708
Epoch 7/20
100/100 [==============================] - 112s 1s/step - loss: 0.2663 - accuracy: 0.8947 - val_loss: 0.7166 - val_accuracy: 0.7243

In [8]:
model=load_model('/content/drive/MyDrive/DATA FOR AI/MV_DATA/VGG_1.h5')
training_model = model.fit(train_set,
                    steps_per_epoch=150,
                    epochs=20,
                    validation_data=test_set,
                    validation_steps=5)
model.save("/content/drive/MyDrive/DATA FOR AI/MV_DATA/VGG_2.h5")

Epoch 1/20
150/150 [==============================] - 152s 960ms/step - loss: 0.2137 - accuracy: 0.9188 - val_loss: 0.4799 - val_accuracy: 0.8306
Epoch 2/20
150/150 [==============================] - 143s 952ms/step - loss: 0.2206 - accuracy: 0.9141 - val_loss: 0.6487 - val_accuracy: 0.7841
Epoch 3/20
150/150 [==============================] - 144s 955ms/step - loss: 0.2032 - accuracy: 0.9224 - val_loss: 0.3393 - val_accuracy: 0.8671
Epoch 4/20
150/150 [==============================] - 145s 966ms/step - loss: 0.2148 - accuracy: 0.9190 - val_loss: 0.2996 - val_accuracy: 0.8804
Epoch 5/20
150/150 [==============================] - 140s 933ms/step - loss: 0.2155 - accuracy: 0.9177 - val_loss: 0.2954 - val_accuracy: 0.8804
Epoch 6/20
150/150 [==============================] - 143s 953ms/step - loss: 0.2095 - accuracy: 0.9187 - val_loss: 0.3596 - val_accuracy: 0.8339
Epoch 7/20
150/150 [==============================] - 142s 942ms/step - loss: 0.2029 - accuracy: 0.9240 - val_loss: 0.3565 -